# Getting Started

In [24]:
import torchhd


In [25]:
d = 10000 # dimensions
fruits = torchhd.random(3, d)
weights = torchhd.level(10, d)
seasons = torchhd.circular(4, d)
var = torchhd.random(3, d)

In [26]:
import torch

weight = torch.tensor([149.0])
# explicit mapping of the fruit weight to an index
w_i = torchhd.value_to_index(weight, 0, 200, 10)
weights[w_i]  # select representation of 149

MAPTensor([[-1., -1.,  1.,  ...,  1., -1.,  1.]])

In [27]:
from torchhd import embeddings

W_emb = embeddings.Level(10, d, low=0, high=200)
# select representation of 149
W_emb(weight)  # same result as weights[w_i]

MAPTensor([[-1., -1.,  1.,  ...,  1., -1.,  1.]])

In [28]:
W_emb(torch.tensor([120.0]))

MAPTensor([[-1.,  1.,  1.,  ...,  1., -1.,  1.]])

In [29]:
f = torchhd.bind(var[0], fruits[0])   # fruit = apple
w = torchhd.bind(var[1], weights[w_i]) # weight = 149
s = torchhd.bind(var[2], seasons[3])   # season = fall
r1 = torchhd.bundle(torchhd.bundle(f, w), s)

In [30]:
from torchhd import functional
from torchhd import embeddings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import MNIST
import torchmetrics


In [31]:
size = 28
DIMENSIONS = 10000
NUM_LEVELS = 10
num_classes = 10
BATCH_SIZE=1

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [33]:
class Model(nn.Module):
    def __init__(self, num_classes, size):
        super(Model, self).__init__()

        self.flatten = torch.nn.Flatten()

        self.position = embeddings.Random(size * size, DIMENSIONS)
        self.value = embeddings.Level(NUM_LEVELS, DIMENSIONS)

        self.classify = nn.Linear(DIMENSIONS, num_classes, bias=False)
        self.classify.weight.data.fill_(0.0)

    def encode(self, x):
        x = self.flatten(x)
        sample_hv = functional.bind(self.position.weight, self.value(x))
        sample_hv = functional.multiset(sample_hv)
        return functional.hard_quantize(sample_hv)

    def forward(self, x):
        enc = self.encode(x)
        logit = self.classify(enc)
        return logit

In [34]:
transform = torchvision.transforms.ToTensor()

train_ds = MNIST("data", train=True, transform=transform, download=False)
train_ld = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

test_ds = MNIST("data", train=False, transform=transform, download=False)
test_ld = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [35]:
im, lbl = next(iter(train_ld))

im = im.to(device)
lbl = lbl.to(device)

model = DNN_HDC(10, 28)

model = model.to(device)

x = model(im)

TypeError: forward() missing 1 required positional argument: 'y'

In [36]:
x.shape

torch.Size([1, 10000])

In [14]:
x = model.flatten(im)

NameError: name 'model' is not defined

In [15]:
model.value(x).shape

NameError: name 'model' is not defined

In [42]:
model.position.weight.shape

torch.Size([784, 10000])

In [44]:
sample_hv = functional.bind(model.position.weight, model.value(x))
sample_hv = functional.multiset(sample_hv)

In [45]:
sample_hv.shape

torch.Size([1, 10000])

In [43]:
samples_hv = model.encode(im)

In [34]:
samples_hv.shape

torch.Size([1, 10000])

In [49]:
model = Model(len(train_ds.classes), 28)
model = model.to(device)

In [50]:
with torch.no_grad():
    for samples, labels in train_ld:
        samples = samples.to(device)
        labels = labels.to(device)

        samples_hv = model.encode(samples)
        model.classify.weight[labels] += samples_hv

    model.classify.weight[:] = F.normalize(model.classify.weight)

In [52]:
model.classify.weight.shape

torch.Size([10, 10000])

In [54]:
with torch.no_grad():
    for samples, labels in test_ld:
        samples = samples.to(device)

        outputs = model(samples)
        predictions = torch.argmax(outputs, dim=-1)
#         accuracy.update(predictions.cpu(), labels)

In [55]:
outputs.shape

torch.Size([1, 10])

In [58]:
predictions = torch.argmax(outputs, dim=-1)

In [59]:
predictions, labels

(MAPTensor([6], device='cuda:0'), tensor([6]))

# Redifining Backward pass

In [16]:
class DNN_HDC(nn.Module):
    def __init__(self, num_classes, size):
        super(DNN_HDC, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        
####
        self.classify = nn.Linear(DIMENSIONS, num_classes, bias=False)
        self.classify.weight.data.fill_(0.0)
        self.position = embeddings.Random(128, DIMENSIONS)
        self.value = embeddings.Level(NUM_LEVELS, DIMENSIONS)
    def forward(self, x, y):
        # Feedforward function
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Reshape for the fully connected layer
        x = torch.relu(self.fc1(x))
        
        x_hdc = HDCModel.apply(x, self.position, self.value)
        with torch.no_grad():
            self.classify.weight[y]+=x_hdc
        return x_hdc

In [17]:
class HDCModel(torch.autograd.Function):
    @staticmethod
    def forward(self, x, position, value):
        sample_hv = functional.bind(position.weight, value(x))
        sample_hv = functional.multiset(sample_hv)
        return functional.hard_quantize(sample_hv)    
    @staticmethod
    def backward(self, x):
        pass

In [18]:
class HDCLossStatic(torch.autograd.Function):
    @staticmethod
    def forward(ctx,hdc_classifier, x_hdc, y_true):
#             hdc_classifier.weight[:] = F.normalize(hdc_classifier.weight)
        y_pred = hdc_classifier(x_hdc)
        return y_pred
    @staticmethod
    def backward():
        pass
        
class HDCLoss(nn.Module):
    def __init__(self):
        super(HDCLoss, self).__init__()

    def forward(self, hdc_classifier, x_hdc, y_true):
        l_hdc = HDCLossStatic.apply(hdc_classifier, x_hdc, y_true)
        print("l_hdc", l_hdc)
        print("Argmax", torch.argmax(l_hdc, dim=-1), "True", y_true)
        ce = nn.CrossEntropyLoss()
        l_hdc = ce(l_hdc, y_true)
        return l_hdc

In [19]:
im, lbl = next(iter(train_ld))

im = im.to(device)
lbl = lbl.to(device)

model = DNN_HDC(10, 28)

model = model.to(device)

x = model(im, lbl)

In [20]:
loss = HDCLoss()
l = loss(model.classify, x, lbl)

l_hdc MAPTensor([[    0.,     0.,     0., 10000.,     0.,     0.,     0.,     0.,
                0.,     0.]], device='cuda:0', grad_fn=<HDCLossStaticBackward>)
Argmax MAPTensor([3], device='cuda:0') True tensor([3], device='cuda:0')


In [39]:
lbl

tensor([7], device='cuda:0')

In [22]:
with torch.no_grad():
    for samples, labels in test_ld:
        samples = samples.to(device)
#         labels = labels.to(device)

        outputs = model(samples, labels)
        predictions = torch.argmax(outputs, dim=-1)
        accuracy.update(predictions.cpu(), labels)

NameError: name 'accuracy' is not defined

In [23]:
print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")

NameError: name 'accuracy' is not defined

In [177]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the CNN model
class CNNMNIST(nn.Module):
    def __init__(self):
        super(CNNMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Reshape for the fully connected layer
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = CNNMNIST()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Load MNIST dataset and create data loaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='data', train=True, transform=transform, download=False)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='data', train=False, transform=transform, download=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    total_accuracy = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
#         print("Ouptus", outputs)
#         print("Lables", labels)
#         break
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        total_accuracy += calculate_accuracy(outputs, labels)

    avg_loss = running_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.2%}")

print("Training finished.")

# Testing the model
model.eval()
total_accuracy = 0.0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        total_accuracy += calculate_accuracy(outputs, labels)

avg_accuracy = total_accuracy / len(test_loader)
print(f"Test Accuracy: {avg_accuracy:.2%}")


KeyboardInterrupt: 